In [1]:
import os
import google.generativeai as genai
from PIL import Image
from datetime import datetime
import pandas as pd
import json




In [2]:
input_data_dir   = '../../data/input/'
input_data_df_dir = '../../data/output/Image_df/'

output_dir = '../../data/output/Image_df/'

input_file_name = 'Crop_Varieties.xlsx'

now = datetime.now()
formatted_date = now.strftime('%Y%m%d%H%M')

In [3]:
params = {
    'model': 'gemini-1.5-pro-002'   # "gemini-1.5-flash"
}

In [4]:
genai.configure(api_key=os.environ['GOOGLE_TENDTEST_KEY'])

In [5]:
def delete_directory_if_exists(directory_path):
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)
        print(f"Deleted directory: {directory_path}")
    else:
        print(f"Directory does not exist: {directory_path}")

def load_crop_data(input_data_dir, input_file_name):
    # Construct the full path to the Excel file
    excel_file_path = os.path.join(input_data_dir, input_file_name)
    
    # Load the Excel file
    excel_data = pd.ExcelFile(excel_file_path)
    
    # Initialize a dictionary to store the crop names and their varieties
    crop_varieties = {}
    
    # Iterate through each sheet in the Excel file
    for sheet_name in excel_data.sheet_names:
        # Read the sheet into a DataFrame without header
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name, header=None)
        
        # Assuming the varieties are listed in the first column
        varieties = df.iloc[:, 0].tolist()
        
        # Store the crop name (sheet name) and its varieties in the dictionary
        crop_varieties[sheet_name] = varieties
    
    return crop_varieties



In [6]:
 # Load crop data from Excel
crop_varieties = load_crop_data(input_data_dir, input_file_name)
crop_varieties

{'Radishes': ['French Breakfast'],
 'Beets': ['Avalanche', 'Badger Flame'],
 'Dill': ['Ella'],
 'Tomatoes': ['Sun Gold'],
 'Peppers': ['Jimmy Nardello'],
 'Cabbage': ['Kaitlin']}

In [7]:
def load_images_from_dataframe(input_dir, crop_name, variety):
    # Construct the path to the DataFrame file
    dataframe_path = os.path.join(input_dir, crop_name, f"{variety}.xlsx")
    
    # Load the DataFrame from the specified path
    df = pd.read_excel(dataframe_path, sheet_name='google_search')
    
    # Extract the 'filename' column into a list called image_paths
    image_paths = df['filename'].tolist()
    
    # Open each image using PIL.Image.open and store them in a list called images
    images = [Image.open(image_path) for image_path in image_paths]
    
    # Return the list of images
    return images

# Example usage

crop_name = 'Beets'
variety = 'Badger Flame'
# crop_name = 'Cabbage'
# variety = 'Kaitlin'

images = load_images_from_dataframe(input_dir = input_data_df_dir, crop_name = crop_name, variety = variety)

In [8]:
# prompt_template = '''
# # Introduction:

# You're an expert vegetable, flower, and plant photographer that can interpret images and output a description that captures 
# lighting, detail, and nuances of realistic still life photography.

# # Step 1:

# You will be provided with several images. Analyze them and select the top 3 images where the {variety} {crop_name} is the primary subject, 
# well-lit, and in focus.  

# Choose images that show the {variety} {crop_name} from multiple angles and with minimal background distractions.

# List the file names or image numbers of these images.

# # Step 2:  Description Synthesis

# From the images you selected, as an expert vegetable, flower, and plant photographer, create detailed descriptions of each image. 
# Pay close attention to the shape, size, color, texture, and any unique markings or patterns on the {variety} {crop_name}. Consider how the 
# light interacts with the surface and describe any shadows or highlights.

# Don't include the individual descriptions in the output, but rather use them as input for Step 3. 

# # Step 3: Reduce to Single Fruit, Vegetable or Flower

# Now that you're an expert in the visual attributes of {variety} {crop_name}, provide a detailed description of a single, typical 
# {variety} {crop_name} on a white background, without describing the variety itself.  

# This description will be used as an input to Imagen3, so tailor your response accordingly. Use evocative language that captures the 
# essence of the {variety} {crop_name} and include descriptive terms related to light, shadow, and texture that Imagen3 can effectively 
# interpret.

# # Step 4: Output Format

# Finally, present your response in a JSON format with the following keys:

# * `CROP_NAME`: equal to the value of {crop_name}
# * `VARIETY`: equal to the value of {variety}
# * `VALID_IMAGES`: the result of Step 1 (list of the image number indices. Indices should start from 1)
# * `PROMPT`: the result of Step 3 (the detailed description)

# Data to follow below:

# [CROP_NAME] = {crop_name}
# [VARIETY] = {variety}
# '''

# prompt = prompt_template.format(crop_name=crop_name, variety=variety)
# print(prompt)

In [9]:
prompt_template = '''
# Introduction:

You're an expert vegetable, flower, and plant photographer that can interpret images and output a description that captures 
lighting, detail, and nuances of realistic still life photography.

# Step 1:

You will be provided with several images. Analyze them and select the top 5 images where the {variety} {crop_name} is the primary subject, 
well-lit, in focus and is not a cross-section or part of the object. 

Choose images that show the {variety} {crop_name} from multiple angles and with minimal background distractions.

List the file names or image numbers of these images.

# Step 2:  Description Synthesis

From the images you selected, as an expert vegetable, flower, and plant photographer, create detailed descriptions of each image. 
Pay close attention to the shape, size, color, texture, and any unique markings or patterns on the {variety} {crop_name}. Consider how the 
light interacts with the surface and describe any shadows or highlights.

Don't include the individual descriptions in the output, but rather use them as input for Step 3. 

# Step 3: Reduce to Single Fruit, Vegetable or Flower

Now that you're an expert in the visual attributes of {variety} {crop_name}, provide a detailed description of a single, typical 
{variety} {crop_name} on a white background, without describing the variety.  

This description will be used as an input to Imagen3, so tailor your response accordingly. Use evocative language that captures the 
essence of the {variety} {crop_name} and include descriptive terms related to shape, size, color, texture, and any unique markings or patterns
as well as light and shadow that Imagen3 can effectively interpret.

Shape is huge component of the description.  Describe the shape of the {variety} {crop_name} in detail.  
Include any unique markings or patterns on the {variety} {crop_name}.

Highlight any differences between the {variety} {crop_name} and other varieties of the same crop so that Imagen3 can distinguish between them.

When appropriate, the description should be of a whole Single Fruit, Vegetable or Flower and not a cross-section or part of the object.

# Step 4: Output Format

Finally, present your response in a JSON format with the following keys:

* `CROP_NAME`: equal to the value of {crop_name}
* `VARIETY`: equal to the value of {variety}
* `VALID_IMAGES`: the result of Step 1 (list of the image number indices. Indices should start from 1)
* `PROMPT`: the result of Step 3 (the detailed description)

Data to follow below:

[CROP_NAME] = {crop_name}
[VARIETY] = {variety}
'''

prompt = prompt_template.format(crop_name=crop_name, variety=variety)
print(prompt)


# Introduction:

You're an expert vegetable, flower, and plant photographer that can interpret images and output a description that captures 
lighting, detail, and nuances of realistic still life photography.

# Step 1:

You will be provided with several images. Analyze them and select the top 5 images where the Badger Flame Beets is the primary subject, 
well-lit, in focus and is not a cross-section or part of the object. 

Choose images that show the Badger Flame Beets from multiple angles and with minimal background distractions.

List the file names or image numbers of these images.

# Step 2:  Description Synthesis

From the images you selected, as an expert vegetable, flower, and plant photographer, create detailed descriptions of each image. 
Pay close attention to the shape, size, color, texture, and any unique markings or patterns on the Badger Flame Beets. Consider how the 
light interacts with the surface and describe any shadows or highlights.

Don't include the individual

In [10]:
# Generate content using the model
model = genai.GenerativeModel(params['model'])
response = model.generate_content([prompt, *images], generation_config=genai.types.GenerationConfig(
    temperature=0.0,
    #max_output_tokens=1024
))


In [11]:

# Access the JSON response from the model
json_response = response.candidates[0].content.parts[0].text

# Strip off the leading and trailing triple backticks and newlines
cleaned_json_response = json_response.strip('```json\n')

# Parse the JSON response with error handling
try:
    response_data = json.loads(cleaned_json_response)
except json.JSONDecodeError as e:
    print(f"JSON decode error: {e}")
    response_data = {}

# Access and print the token usage
token_usage = response.usage_metadata

# Prepare data for the new sheet
data = {
    'CROP_NAME': [response_data['CROP_NAME']],
    'VARIETY': [response_data['VARIETY']],
    'VALID_IMAGES': [response_data['VALID_IMAGES']],
    'PROMPT': [response_data['PROMPT']],
    'PROMPT_TOKENS': [token_usage.prompt_token_count],
    'COMPLETION_TOKENS': [token_usage.candidates_token_count],
    'TOTAL_TOKENS': [token_usage.total_token_count]
}

# Convert data to DataFrame
df_output = pd.DataFrame(data)

# Write to a new sheet in the variety.xlsx file
output_file = os.path.join(output_dir, crop_name, f"{variety}.xlsx")
with pd.ExcelWriter(output_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_output.to_excel(writer, sheet_name='gemini_output', index=False)

print("Results written to gemini_output sheet in variety.xlsx")
display(df_output)
print(df_output['PROMPT'][0])

Results written to gemini_output sheet in variety.xlsx


,CROP_NAME,VARIETY,VALID_IMAGES,PROMPT,PROMPT_TOKENS,COMPLETION_TOKENS,TOTAL_TOKENS
0,Beets,Badger Flame,"[1, 3, 7]",A single Badger Flame beet rests on a bright w...,3151,250,3401


A single Badger Flame beet rests on a bright white background, its smooth skin gleaming under the light.  It is a vibrant, almost incandescent, orange-red, a color that deepens to ruby where the beet curves away from the light source.  The shape is a slightly elongated globe, wider at the top and tapering gently towards the root end.  This rounded, almost squat form distinguishes it from the more elongated shape of other beet varieties.  The beet's surface is subtly textured, not quite glossy, but with a soft sheen that reflects the light in delicate highlights.  Fine, almost invisible root hairs cling near the tapered bottom, where the beet narrows to meet the remnants of its taproot.  A faint network of pale, slightly raised veins traces a delicate pattern across the skin, adding a touch of complexity to its otherwise smooth surface.  The overall impression is one of vibrant color and earthy elegance, a jewel-toned root vegetable radiating a warm, inner glow.
